# Template for creating BLC classifier

### From Forlin et al 

# Imports:

In [1]:
import anndata as ad
import scanpy as sc
import numpy as np
import pickle
import pandas as pd
from sklearn.preprocessing import quantile_transform
from tqdm import tqdm
import pickle
import os
import time
from BLC_classification_lib import MultiClassWrapper, calc_emd, OneVsOneWrapper 

# Create new classifier

In [2]:
def get_train_data(data_path, label_name = 'CellType'):
    
    train_ad = sc.read(data_path)
    
    #Delete MT, RPS, RPL and LINC genes
    searchfor = ['MT-', 'RPS', 'RPL', 'LINC00']
    train_ad = train_ad[:, ~train_ad.var.index.str.contains('|'.join(searchfor))]
    sc.pp.normalize_total(train_ad, target_sum = 1e4)
    
    train_ad_df = train_ad.to_df()
    trained_scaled = quantile_transform(train_ad_df, subsample = len(train_ad), output_distribution= 'uniform', random_state = 0)
    trained_scaled = pd.DataFrame(trained_scaled, index = train_ad_df.index, columns = train_ad_df.columns)

    train_labels = train_ad.obs[label_name].astype(str)
    train_classes = list(set(train_labels))
    
    return trained_scaled, train_labels, train_classes

In [3]:
def get_emd_genes_and_filter_df(train_samples_scaled, train_labels, train_classes, balanced_ct_genes, max_genes, minimum_genes, thres):
    
    emd = {}

    '''Genes based on 1v1 expressions'''
    for tc1 in train_classes:
        for tc2 in train_classes:
            if tc1==tc2:
                break 
            tc1_data = train_samples_scaled[train_labels == tc1]
            tc2_data = train_samples_scaled[train_labels == tc2]
            emd[(tc1, tc2)] = calc_emd(tc1_data, tc2_data)

    ct_emds_specific = {}

    print("(in get_emd_genes_and_filter_df) - Finding genes with highest EMD")

    for k in tqdm(emd.keys()):
            tc1 = k[0]
            tc2 = k[1]

            emd_temp = emd[k]

            ct_emds_specific[k] = {}
            #Here we are getting @max_genes genes from each cell type, to balance out what marker genes we are looking at. We still need to pass a threshold of emd to include that marker gene.
            tc1_expression = np.median(train_samples_scaled.loc[train_labels[train_labels == tc1].index], axis = 0) 
            tc2_expression = np.median(train_samples_scaled.loc[train_labels[train_labels == tc2].index], axis = 0) 
            tc1_expression = pd.Series(data = tc1_expression, index = train_samples_scaled.columns)
            tc2_expression = pd.Series(data = tc2_expression, index = train_samples_scaled.columns)

            tc1_expression_filt = tc1_expression[tc1_expression > tc2_expression]
            tc2_expression_filt = tc2_expression[tc2_expression > tc1_expression]

            if balanced_ct_genes:
                emds_tc1 = emd_temp.loc[tc1_expression_filt.index].nlargest(int(max_genes)).index
                emds_tc2 = emd_temp.loc[tc2_expression_filt.index].nlargest(int(max_genes)).index
                emds_tc1_filt = emds_tc1[emds_tc1 >= thres]
                emds_tc2_filt = emds_tc2[emds_tc2 >= thres]
                
                #if we can't find enough of the good marker genes, select the minimum number of genes with the highest EMD
                if len(emds_tc1_filt) < minimum_genes:
                    emds_tc1_filt = emds_tc1.nlargest(minimum_genes)
                ct_emds_specific[k][tc1] = emds_tc1_filt

                if len(emds_tc2_filt) < minimum_genes:
                    emds_tc2_filt = emds_tc2.nlargest(minimum_genes)
                ct_emds_specific[k][tc2] = emds_tc2_filt
                

            else:
                emd_temp_filt = emd_temp[emd_temp >= thres]
                emds_tc1 = emd_temp.loc[tc1_expression_filt.index]
                emds_tc2 = emd_temp.loc[tc2_expression_filt.index]
                
                #if we can't find enough of the good marker genes, select the minimum number of genes with the highest EMD
                if len(emd_temp_filt) < minimum_genes:
                    genes_to_include = emd_temp.nlargest(minimum_genes).index
                    ct_emds_specific[k][tc1] = emds_tc1.index[emds_tc1.index.isin(genes_to_include)]
                    ct_emds_specific[k][tc2] = emds_tc2.index[emds_tc2.index.isin(genes_to_include)]
                else:
                    genes_to_include = emd_temp_filt.nlargest(int(max_genes)*2).index
                    ct_emds_specific[k][tc1] = emds_tc1.index[emds_tc1.index.isin(genes_to_include)]
                    ct_emds_specific[k][tc2] = emds_tc2.index[emds_tc2.index.isin(genes_to_include)]

    
    all_high_emd_genes = []    
    for key_1 in ct_emds_specific.keys():
        for key_2 in ct_emds_specific[key_1].keys():
            if len(ct_emds_specific[key_1][key_2]) == 0:
                continue
            else:
                good_genes = ct_emds_specific[key_1][key_2].values
                all_high_emd_genes += list(good_genes)
                all_high_emd_genes = list(set(all_high_emd_genes))

    all_high_emd_genes = np.array(all_high_emd_genes)
    train_samples_scaled_filt = train_samples_scaled.loc[:, all_high_emd_genes]
    return train_samples_scaled_filt, ct_emds_specific

In [ ]:
data_path = 'insert data path to data train on'
label_name = 'CellType' #columns name in the h5ad.obs-dataframe which have the correct classifications of the cells


train_samples_scaled, train_labels, train_classes = get_train_data(data_path, label_name)
print('Done getting training data')

#Next, we get the EMD marker genes and filter our original data
balanced_ct_genes = False #If we want the same amount of marker genes from both cell types
max_genes = 70 #Maximum number of genes
minimum_genes = 5 #minimum number of genes
thres = 0.4 #Threshold for emd-distance

train_samples_scaled_filt, ct_emds_specific = get_emd_genes_and_filter_df(train_samples_scaled, train_labels, train_classes, balanced_ct_genes, max_genes, minimum_genes, thres)
print('Done getting emd marker genes and filtering training data')


#Lastly, we train the classifier with the following parameters
max_depth = 50
n_estimators = 200
min_sample_leaf = 20
max_features = 'sqrt' 
include_naive = True #Include Bayesian classifier as root node in Random Forest or not
class_weights = None

#For saving in classifier
param_dic = {'maxDepth': max_depth, 'maxGenes':max_genes, 'minimum_genes': minimum_genes, 'emd_threshold':thres, 'class_weights':class_weights, 'balanced_ct_genes':balanced_ct_genes,
                'min_sample_leaf':min_sample_leaf, 'n_estimators':n_estimators, 'max_features':max_features, 'emd_threshold': thres }

clf = OneVsOneWrapper(MultiClassWrapper(max_depth = max_depth, n_estimators=n_estimators, min_samples_leaf = min_sample_leaf, max_features = max_features, include_naive=include_naive, class_weights = class_weights, parameters = param_dic))
clf.set_specific_marker_dic(ct_emds_specific)
clf.fit(train_samples_scaled_filt, train_labels)

print("Done training!")

#Save classifier:
save_loc = './'
savestr ='BLC_classifier_maxdepth' + str(max_depth) + '_maxgenes' + str(max_genes) + 'n_estimators' + str(n_estimators)
with open(save_loc + savestr +'.pickle', 'wb') as handle:
    pickle.dump(clf, handle, protocol= pickle.HIGHEST_PROTOCOL)

print('saved classifier at location ' + save_loc + savestr + '.pickle')



